### LLMRouterChain

A bit complicated than previous chains, just understand the workflow and template.

![image](../docs/llm_router_chain_components.png)

#### MultiPrompt Chain

In [16]:
from dotenv import find_dotenv, load_dotenv
import os
from langchain_openai import AzureChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.prompts import PromptTemplate

# Import chains, router chains and template
from langchain.chains import LLMChain, MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router import MultiRouteChain
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

load_dotenv(find_dotenv())

model = AzureChatOpenAI(
    openai_api_type="azure",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    openai_api_key=os.getenv("AZURE_OPENAI_API_KEY")
)

In [3]:
hr_template="""You are a very smart human resource persion. \
You are great at answering questions about human resoures in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{input}"""

law_template="""Your are a very smart laywer. \
You are great at answering questions about the law in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{input}"""

prompt_infos = [
    {
        'name': 'human resources',
        'description': 'Answer human resources questions',
        'template': hr_template
    },{
        'name': 'law',
        'description': 'Answer legal questions',
        'template': law_template
    }
]

In [20]:
destination_chains = {}

for _info in prompt_infos:
    name = _info['name']
    prompt_template = _info['template']
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm = model, prompt = prompt)
    destination_chains[name] = chain

In [7]:
default_prompt = ChatPromptTemplate.from_template('{input}')
default_chain = LLMChain(llm = model, prompt = default_prompt)

In [8]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [13]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destination_str = "\n".join(destinations)
print(destination_str)

human resources: Answer human resources questions
law: Answer legal questions


In [15]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations = destination_str)
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
human resources: Answer human resources questions
law: Answer legal questions

<< INPUT >>
{input}

<

In [21]:
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=['input'],
    output_parser=RouterOutputParser()
)

router_chain = LLMRouterChain.from_llm(llm=model, prompt=router_prompt)

chain = MultiPromptChain(
    router_chain = router_chain,
    destination_chains = destination_chains,
    default_chain = default_chain,
    verbose = True
)

chain.run("How do intellectual property laws address the use of AI-generated content?")



> Entering new MultiPromptChain chain...
law: {'input': 'How do intellectual property laws address the use of AI-generated content?'}
> Finished chain.


'Intellectual Property (IP) laws as they relate to AI-generated content are complex and rapidly evolving. Currently, there is no international consensus on whether AI-generated content can be protected by copyright. In most jurisdictions, there are two key requirements for copyright protection: originality and human authorship. \n\nThe originality requirement is usually met when the work is not copied from another work and embodies the minimal degree of creativity. AI-generated content can arguably meet this requirement.\n\nHuman authorship, however, is a more difficult requirement for AI-generated content. In many jurisdictions, copyright protection only extends to works created by humans. \n\nIn the United States, for example, the U.S. Copyright Office has declared that it will only register works created by human beings. This means that works created by an AI, without any creative input or intervention from a human, would likely not be eligible for copyright protection.\n\nIn the UK

In [22]:
chain.run("How do you handle conflict between team members?")



> Entering new MultiPromptChain chain...
human resources: {'input': 'How do you handle conflict between team members?'}
> Finished chain.


'Handling conflict between team members involves several steps:\n\n1. Identify the Problem: The first step in resolving any conflict is to understand what the issue is. Talk to each team member involved individually to get their perspective.\n\n2. Foster Open Communication: Arrange a meeting with those involved in the conflict. Encourage open and honest communication, where each party can share their side of the story without interruption.\n\n3. Find Common Ground: Try to identify any points of agreement, no matter how small. This can help to build a sense of unity and cooperation.\n\n4. Develop Solutions: Once the problem is clearly defined, and all parties have had a chance to express their feelings, work with the team members to develop a solution. This should ideally be a compromise that addresses the main concerns of all parties involved.\n\n5. Implement and Monitor the Solution: Once a solution has been agreed upon, it should be put into action. Continue to monitor the situation 